In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
train_path = "Embedded/trainset_embedded_nltk_full.csv"
test_path = "Embedded/testset_embedded_nltk_full.csv"
#train_path = "/content/drive/MyDrive/Code/DSAA/data/embedded/trainset_embedded_nltk_full.csv"
#test_path = "/content/drive/MyDrive/Code/DSAA/data/embedded/testset_embedded_nltk_full.csv"
split_ratio = 0.1
# weighted or nonweighted
postag_type = "weighted"

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# Import and Pre-process

## Import

In [3]:
# Load
data = pd.read_csv(train_path, index_col = 0)
data

,$1,VBD1,UH1,NNS1,JJS1,NNP1,''1,PDT1,IN1,JJR1,...,RB2,WP$2,CD2,DT2,TO2,MD2,PRP2,WRB2,WP2,label
0,0.0,2.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0,...,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1.0,0.0,41.0,4.0,0.0,0.0,1.0,0.0,0.0,0
2,0.0,4.0,0.0,30.0,2.0,0.0,0.0,0.0,49.0,0.0,...,1.0,0.0,6.0,2.0,1.0,0.0,1.0,0.0,0.0,1
3,0.0,13.0,0.0,25.0,0.0,2.0,0.0,0.0,30.0,1.0,...,0.0,0.0,2.0,3.0,1.0,0.0,1.0,0.0,0.0,0
4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,...,2.0,0.0,7.0,7.0,0.0,0.0,2.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948227,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,...,2.0,0.0,6.0,5.0,0.0,0.0,2.0,1.0,1.0,1
948228,0.0,6.0,0.0,21.0,0.0,0.0,0.0,0.0,28.0,0.0,...,3.0,0.0,5.0,6.0,2.0,0.0,2.0,0.0,0.0,0
948229,0.0,6.0,0.0,4.0,0.0,0.0,0.0,0.0,15.0,0.0,...,0.0,0.0,5.0,6.0,0.0,0.0,3.0,0.0,0.0,0
948230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,2.0,0.0,5.0,4.0,0.0,0.0,1.0,0.0,0.0,0


In [4]:
# Load test
test = pd.read_csv(test_path)
test

,$1,VBD1,UH1,NNS1,JJS1,NNP1,''1,PDT1,IN1,JJR1,...,PRP$2,RB2,WP$2,CD2,DT2,TO2,MD2,PRP2,WRB2,WP2
0,0.0,3.0,0.0,9.0,1.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,8.0,4.0,0.0,0.0,1.0,0.0,0.0
1,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,12.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,3.0,0.0,13.0,6.0,1.0,1.0,1.0,0.0,0.0
4,0.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,20.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238359,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,3.0,0.0,4.0,10.0,2.0,0.0,2.0,1.0,0.0
238360,0.0,2.0,0.0,25.0,0.0,0.0,0.0,0.0,46.0,1.0,...,0.0,0.0,0.0,14.0,4.0,1.0,0.0,1.0,0.0,0.0
238361,0.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,9.0,1.0,...,0.0,3.0,0.0,8.0,25.0,11.0,0.0,5.0,3.0,0.0
238362,0.0,2.0,0.0,19.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,2.0,0.0,2.0,6.0,0.0,1.0,3.0,0.0,0.0


In [5]:
#test_label = pd.read_csv("/content/drive/MyDrive/Code/DSAA/data/test.csv")
test_label = pd.read_csv("test.csv")

In [6]:
test_id = test_label['id']

## Split train test

In [7]:
y = np.array(data['label'])
y

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [8]:
def de_weighted(x):
  if x > 0:
    return 1
  if x == 0:
    return 0


postag = list(data.columns)
postag.remove("label")

X = np.array(data[postag])

if postag_type != "weighted":
  deweight = np.vectorize(de_weighted)
  X = deweight(X)

X

array([[0., 2., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 4., 0., ..., 1., 0., 0.],
       ...,
       [0., 6., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 7., 0., ..., 0., 0., 0.]])

**Splitting**

In [9]:
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=split_ratio, random_state=42)

# Models

## KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [13]:
y_pred = neigh.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93     51198
           1       1.00      0.83      0.91     43626

    accuracy                           0.92     94824
   macro avg       0.94      0.92      0.92     94824
weighted avg       0.93      0.92      0.92     94824



## LogisticRegression

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', verbose = True)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\takhu\anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.0s finished


In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.78      0.69     51198
           1       0.64      0.45      0.53     43626

    accuracy                           0.63     94824
   macro avg       0.63      0.62      0.61     94824
weighted avg       0.63      0.63      0.62     94824



## Decision Tree

In [ ]:
from sklearn import tree

decision_tree = tree.DecisionTreeClassifier()
decision_tree = decision_tree.fit(X_train, y_train)

y_pred = decision_tree.predict(X_test)
# decision_tree = decision_tree.fit(X, y)

In [ ]:
print(classification_report(y_test, y_pred, digits = 8))

              precision    recall  f1-score   support

           0  0.99986328 0.99992187 0.99989258     51198
           1  0.99990831 0.99983955 0.99987392     43626

    accuracy                      0.99988400     94824
   macro avg  0.99988579 0.99988071 0.99988325     94824
weighted avg  0.99988400 0.99988400 0.99988400     94824



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=150, random_state=0)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, digits = 16))

                  precision    recall  f1-score   support

               0  0.9999804683685228 1.0000000000000000 0.9999902340888893     51198
               1  1.0000000000000000 0.9999770778893321 0.9999885388133087     43626

        accuracy                      0.9999894541466295     94824
       macro avg  0.9999902341842615 0.9999885389446661 0.9999893864510990     94824
    weighted avg  0.9999894543526073 0.9999894541466295 0.9999894541376904     94824



## Extra trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

forest = ExtraTreesClassifier(n_estimators=150, random_state=0)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, digits = 16))

                  precision    recall  f1-score   support

               0  0.9999804683685228 1.0000000000000000 0.9999902340888893     51198
               1  1.0000000000000000 0.9999770778893321 0.9999885388133087     43626

        accuracy                      0.9999894541466295     94824
       macro avg  0.9999902341842615 0.9999885389446661 0.9999893864510990     94824
    weighted avg  0.9999894543526073 0.9999894541466295 0.9999894541376904     94824



## XGBoost

In [ ]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
from skopt import BayesSearchCV

In [ ]:
import xgboost as xgb
xgb_cl = xgb.XGBClassifier(eta=0.5, max_depth=40, n_estimators=250)
xgb_cl.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.5, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=40,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=250, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [ ]:
y_pre_xg = xgb_cl.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pre_xg, digits = 16))

                  precision    recall  f1-score   support

               0  0.9999023494717106 1.0000000000000000 0.9999511723518324     51198
               1  1.0000000000000000 0.9998853894466603 0.9999426914392473     43626

        accuracy                      0.9999472707331477     94824
       macro avg  0.9999511747358554 0.9999426947233301 0.9999469318955398     94824
    weighted avg  0.9999472758821885 0.9999472707331477 0.9999472705095516     94824



## LightBGM

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier()

In [ ]:
y_pre_lg = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pre_lg, digits = 16))

                  precision    recall  f1-score   support

               0  0.9973701127929402 1.0000000000000000 0.9986833250431577     51198
               1  1.0000000000000000 0.9969055150598267 0.9984503598608767     43626

        accuracy                      0.9985763097949886     94824
       macro avg  0.9986850563964701 0.9984527575299134 0.9985668424520172     94824
    weighted avg  0.9985800539396456 0.9985763097949886 0.9985761439598646     94824



## MLP

In [ ]:
X_train.shape

(853408, 36)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Normalization
from keras.optimizers import Adam

#mlp = Sequential()
#mlp.add(Dense(128, activation='relu', input_shape=(36,)))
#mlp.add(Dense(64, activation='relu'))
#mlp.add(Dense(1, activation='sigmoid'))

In [ ]:
mlp = Sequential()
mlp.add(Flatten(input_shape=(36,)))
mlp.add(Normalization())
#mlp.add(Dropout(0.2))
mlp.add(Dense(128, activation='relu'))
mlp.add(Dense(64, activation='relu'))
mlp.add(Dense(32, activation='relu'))
mlp.add(Dense(1, activation='sigmoid'))

In [ ]:
mlp.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [ ]:
mlp.compile(loss='binary_crossentropy',
optimizer=Adam(learning_rate=0.001),
metrics=['accuracy'])

In [ ]:
mlp.fit(X_train, y_train,epochs=30, batch_size=32, verbose=1)

Epoch 1/30
26669/26669 [==============================] - 59s 2ms/step - loss: 0.1334 - accuracy: 0.9400
Epoch 2/30
26669/26669 [==============================] - 57s 2ms/step - loss: 0.0296 - accuracy: 0.9906
Epoch 3/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0201 - accuracy: 0.9935
Epoch 4/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0163 - accuracy: 0.9950
Epoch 5/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0132 - accuracy: 0.9958
Epoch 6/30
26669/26669 [==============================] - 54s 2ms/step - loss: 0.0113 - accuracy: 0.9965
Epoch 7/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0106 - accuracy: 0.9967
Epoch 8/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0094 - accuracy: 0.9972
Epoch 9/30
26669/26669 [==============================] - 56s 2ms/step - loss: 0.0091 - accuracy: 0.9973
Epoch 10/30
26669/26669 [==============================

In [ ]:
mlp.save("/content/drive/MyDrive/Code/DSAA/models/mlp.h5")

In [ ]:
y_pred = mlp.predict(X_test)

2964/2964 [==============================] - 8s 3ms/step


In [ ]:
def to_class(x):
  if x>0.5:
    return 1
  else:
    return 0

In [ ]:
y_pred_mlp = [to_class(x) for x in y_pred]
y_pred = y_pred_mlp

In [ ]:
print(classification_report(y_test, y_pred, digits = 8))

              precision    recall  f1-score   support

           0  0.99719424 0.99962889 0.99841008     51198
           1  0.99956323 0.99669922 0.99812917     43626

    accuracy                      0.99828103     94824
   macro avg  0.99837873 0.99816405 0.99826962     94824
weighted avg  0.99828415 0.99828103 0.99828084     94824



# Predict Testset

In [ ]:
# y_pred_on_test = decision_tree.predict(np.array(test))
y_pred_on_test = forest.predict(np.array(test))
submission = pd.DataFrame(np.array([list(test_id), list(y_pred_on_test)]).T)
submission.columns = ["id", "label"]

In [ ]:
# y_pred_on_test = decision_tree.predict(np.array(test))
y_pred_on_test = forest.predict(np.array(test))
submission = pd.DataFrame(np.array([list(test_id), list(y_pred_on_test)]).T)
submission.columns = ["id", "label"]

In [ ]:
# y_pred_on_test = decision_tree.predict(np.array(test))
y_pred_on_test = xgb_cl.predict(np.array(test))
y_pred_on_test_xgb = [to_class(x) for x in y_pred_on_test]
submission = pd.DataFrame(np.array([list(test_id), list(y_pred_on_test_xgb)]).T)
submission.columns = ["id", "label"]

In [ ]:
submission.to_csv("/content/drive/MyDrive/Code/DSAA/sub/submission25_xgb_nltkfull.csv", index=False)